# Metal Abundance Evolution

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from multiprocessing import Pool

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np

In [3]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.parser import parse

In [4]:
figure_setup()
settings = Settings()

In [5]:
def read_data(simulation: str) -> pd.DataFrame:
    s = Snapshot(simulation=simulation, loadonlytype=[0, 4])
    s.add_metal_abundance(of="Fe", to='H')
    s.add_metal_abundance(of='O', to='H')
    s.add_metal_abundance(of='O', to="Fe")
    s.add_extra_coordinates()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_gas = s.type == 0
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)
    full_mask = is_main_obj & (is_real_star | is_gas)

    props = {
        "[Fe/H]": s.metal_abundance["Fe/H"][full_mask],
        "[O/H]": s.metal_abundance["O/H"][full_mask],
        "[O/Fe]": s.metal_abundance["O/Fe"][full_mask],
        "ParticleType": s.type[full_mask],
        "SphericalRadius_ckpc": s.r[full_mask]
        }
    
    df = pd.DataFrame(props)
    df[~np.isfinite(df)] = np.nan
    df.dropna(inplace=True)

    return df

In [6]:
def calculate_evolution_data(simulation: str) -> pd.DataFrame:
    settings = Settings()
    _, rerun, resolution = parse(simulation)
    abundances = [("Fe", "H"), ("O", "H"), ("O", "Fe")]

    time = pd.read_csv(f"../results/{simulation}/temporal_data.csv")
    props = {
        "Time_Gyr": time["Time_Gyr"],
        "[Fe/H]_Type4_Mean": np.nan * np.ones(len(time)),
        "[O/H]_Type4_Mean": np.nan * np.ones(len(time)),
        "[O/Fe]_Type4_Mean": np.nan * np.ones(len(time)),
        "[Fe/H]_Type0_Mean": np.nan * np.ones(len(time)),
        "[O/H]_Type0_Mean": np.nan * np.ones(len(time)),
        "[O/Fe]_Type0_Mean": np.nan * np.ones(len(time)),
        "[Fe/H]_Type0_30ckpc_Mean": np.nan * np.ones(len(time)),
        "[O/H]_Type0_30ckpc_Mean": np.nan * np.ones(len(time)),
        "[O/Fe]_Type0_30ckpc_Mean": np.nan * np.ones(len(time)),
    }

    for i in range(len(time)):
        if i >= settings.first_snap:
            data = read_data(f"{simulation}_s{i}")
            for part_type in [0, 4]:
                is_part_type = data["ParticleType"] == part_type
                for m1, m2 in abundances:
                    props[f"[{m1}/{m2}]_Type{part_type}_Mean"][i] \
                        = np.nanmean(data[f"[{m1}/{m2}]"][is_part_type])

            # Add average for gas inside 30 ckpc
            max_radius = 30  # ckpc
            props["[Fe/H]_Type0_30ckpc_Mean"][i] = np.nanmean(
                data["[Fe/H]"][
                    (data["ParticleType"] == 0) \
                        & (data["SphericalRadius_ckpc"] <= max_radius)]
            )
            props["[O/H]_Type0_30ckpc_Mean"][i] = np.nanmean(
                data["[O/H]"][
                    (data["ParticleType"] == 0) \
                        & (data["SphericalRadius_ckpc"] <= max_radius)]
            )
            props["[O/Fe]_Type0_30ckpc_Mean"][i] = np.nanmean(
                data["[O/Fe]"][
                    (data["ParticleType"] == 0) \
                        & (data["SphericalRadius_ckpc"] <= max_radius)]
            )


    df = pd.DataFrame(props)
    df.to_csv(f"../results/{simulation}/metallicity_evolution.csv")

In [22]:
# Create all data frames
simulations = [f"au{i}_or_l4" for i in settings.groups["Included"]]
Pool().map(calculate_evolution_data, simulations)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [7]:
def make_plot_for_simulation(simulation: str) -> None:
    galaxy = parse(simulation)[0]
    label = f"Au{galaxy}"
    df = pd.read_csv(f"../results/{simulation}/metallicity_evolution.csv")

    fig, ax = plt.subplots(figsize=(2.5, 2.5))

    ax.tick_params(which='both', direction="in")
    ax.set_axisbelow(True)
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")

    ax.set_xlim(0, 14)
    ax.set_xticks([2, 4, 6, 8, 10, 12])
    ax.set_xlabel("Time [Gyr]")

    ax.set_ylim(-2, 1.5)
    ax.set_yticks([-1.5, -1, -0.5, 0, 0.5, 1.0])
    ax.set_ylabel("Abundance")

    ax.plot(df["Time_Gyr"], df["[Fe/H]_Type4_Mean"],
            c="red", ls="-", lw=1.0, zorder=12, label=r"[Fe/H]$_\star$")
    ax.plot(df["Time_Gyr"], df["[O/H]_Type4_Mean"],
            c="blue", ls="-", lw=1.0, zorder=12, label=r"[O/H]$_\star$")
    ax.plot(df["Time_Gyr"], df["[O/Fe]_Type4_Mean"],
            c="green", ls="-", lw=1.0, zorder=12, label=r"[O/Fe]$_\star$")
    ax.plot(df["Time_Gyr"], df["[Fe/H]_Type0_Mean"],
            c="red", ls=(0, (3, 1)), lw=0.75, zorder=12,
            label=r"[Fe/H]$_\mathrm{gas}$")
    ax.plot(df["Time_Gyr"], df["[O/H]_Type0_Mean"],
            c="blue", ls=(0, (3, 1)), lw=0.75, zorder=12,
            label=r"[O/H]$_\mathrm{gas}$")
    ax.plot(df["Time_Gyr"], df["[O/Fe]_Type0_Mean"],
            c="green", ls=(0, (3, 1)), lw=0.75, zorder=12,
            label=r"[O/Fe]$_\mathrm{gas}$")
    ax.plot(df["Time_Gyr"], df["[Fe/H]_Type0_30ckpc_Mean"],
            c="red", ls=(0, (1, 1)), lw=0.75, zorder=12,
            label=r"[Fe/H]$_\mathrm{gas}^{30~\mathrm{ckpc}}$")
    ax.plot(df["Time_Gyr"], df["[O/H]_Type0_30ckpc_Mean"],
            c="blue", ls=(0, (1, 1)), lw=0.75, zorder=12,
            label=r"[O/H]$_\mathrm{gas}^{30~\mathrm{ckpc}}$")
    ax.plot(df["Time_Gyr"], df["[O/Fe]_Type0_30ckpc_Mean"],
            c="green", ls=(0, (1, 1)), lw=0.75, zorder=12,
            label=r"[O/Fe]$_\mathrm{gas}^{30~\mathrm{ckpc}}$")
    
    ax.legend(loc="lower right", framealpha=0.0, fontsize=4.0,
                bbox_to_anchor=(0.99, 0.01), ncol=3)

    ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$",
            size=8.0, transform=ax.transAxes, ha='left', va='top')

    fig.savefig(
        f"../images/metallicity_evolution/{simulation}.pdf")
    plt.close(fig)

In [11]:
def make_plot_for_sample() -> None:
    fig = plt.figure(figsize=(7, 8))
    gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flatten():
        ax.grid(True, ls='-', lw=0.25, c="gainsboro")
        ax.tick_params(which='both', direction="in")
        if ax == axs[-1, -1]: ax.axis("off")
        ax.set_xlim(0, 14)
        ax.set_xticks([2, 4, 6, 8, 10, 12])
        ax.set_ylim(-2.0, 1.5)
        ax.set_yticks([-1.5, -1, -0.5, 0, 0.5, 1.0])
        ax.set_axisbelow(True)
        if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
            ax.set_xlabel("Time [Gyr]")
            ax.tick_params(labelbottom=True)
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel("Abundance")

    for i, galaxy in enumerate(settings.groups["Included"]):
        ax = axs.flatten()[i]
        simulation = f"au{galaxy}_or_l4"
        label = f"Au{galaxy}"
        df = pd.read_csv(f"../results/{simulation}/metallicity_evolution.csv")

        ax.plot(df["Time_Gyr"], df["[Fe/H]_Type4_Mean"],
                c="red", ls="-", lw=1.0, zorder=12, label=r"[Fe/H]$_\star$")
        ax.plot(df["Time_Gyr"], df["[O/H]_Type4_Mean"],
                c="blue", ls="-", lw=1.0, zorder=12, label=r"[O/H]$_\star$")
        ax.plot(df["Time_Gyr"], df["[O/Fe]_Type4_Mean"],
                c="green", ls="-", lw=1.0, zorder=12, label=r"[O/Fe]$_\star$")
        ax.plot(df["Time_Gyr"], df["[Fe/H]_Type0_Mean"],
                c="red", ls=(0, (3, 1)), lw=0.75, zorder=12,
                label=r"[Fe/H]$_\mathrm{gas}$")
        ax.plot(df["Time_Gyr"], df["[O/H]_Type0_Mean"],
                c="blue", ls=(0, (3, 1)), lw=0.75, zorder=12,
                label=r"[O/H]$_\mathrm{gas}$")
        ax.plot(df["Time_Gyr"], df["[O/Fe]_Type0_Mean"],
                c="green", ls=(0, (3, 1)), lw=0.75, zorder=12,
                label=r"[O/Fe]$_\mathrm{gas}$")
        ax.plot(df["Time_Gyr"], df["[Fe/H]_Type0_30ckpc_Mean"],
                c="red", ls=(0, (1, 1)), lw=0.75, zorder=12,
                label=r"[Fe/H]$_\mathrm{gas}^{30~\mathrm{ckpc}}$")
        ax.plot(df["Time_Gyr"], df["[O/H]_Type0_30ckpc_Mean"],
                c="blue", ls=(0, (1, 1)), lw=0.75, zorder=12,
                label=r"[O/H]$_\mathrm{gas}^{30~\mathrm{ckpc}}$")
        ax.plot(df["Time_Gyr"], df["[O/Fe]_Type0_30ckpc_Mean"],
                c="green", ls=(0, (1, 1)), lw=0.75, zorder=12,
                label=r"[O/Fe]$_\mathrm{gas}^{30~\mathrm{ckpc}}$")

        ax.text(x=0.05, y=0.95,
                s=r"$\texttt{" + label + "}$",
                size=6.0, transform=ax.transAxes,
                ha='left', va='top',
                )

        if galaxy == 24:
            ax.legend(loc="upper center", framealpha=0.0, fontsize=5.0,
                      ncol=2, bbox_to_anchor=(0.5, -0.5))

    fig.savefig(
        f"../images/metallicity_evolution/included.pdf")
    plt.close(fig)

In [12]:
# Create figures
make_plot_for_simulation("au6_or_l4")
make_plot_for_sample()